In [34]:
import os
import pandas as pd
import warnings
import xarray as xr

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

path = './old_data'

### 수질지수 데이터 전처리

In [2]:
def fix_input_errors(df, column_name, error_value, correct_value):
    """
    해당 데이터프레임의 특정 열에서 발견된 입력 오류 수정
    
    Parameters:
    df (pandas.DataFrame): 수정할 데이터프레임
    column_name (str): 수정할 열의 이름
    error_value (str): 잘못된 값
    correct_value (str or float): 올바른 값
    """
    cond = df[column_name] == error_value
    df.loc[cond, column_name] = correct_value

def date_ocean(df):
    df['년'] = df['년'].astype(str)
    df['월'] = df['월'].astype(str).str.zfill(2)
    df['조사일'] = df['조사일'].astype(str).apply(lambda x: x.split()[0].split('-')[-1].split('.')[0].zfill(2))
    df['조사일자'] = df['년'] + '-' + df['월'] + '-' + df['조사일'] + ' ' + df['조사기간'].astype(str)
    df.sort_values(['조사일자'], ascending=True, inplace=True)
    return df

def invalid(df):
    # '일자' 열에서 YYYY-MM-DD 형식과 YYYY-MM-DD-HH:MM:SS 형식이 아닌 값 확인
    invalid_dates = df['조사일자'][~(df['조사일자'].str.match(r'^\d{4}-\d{2}-\d{2}$') | df['조사일자'].str.match(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$') | df['조사일자'].str.match(r'^\d{4}-\d{1,2}-\d{1,2} \d{2}:\d{2}:\d{2}$'))]
    return invalid_dates

def ocean_rank(df):
    if '조사일자' not in df.columns:
        df = date_ocean(df)
    return df[['조사일자', '생태구', '기상', '표층수온', '저층수온', '표층염분', '저층염분', '표층pH', '저층pH', '표층DO', '저층DO', '표층chl', '저층Chl-a', '수질지수']]

# 일자 오입력 처리 함수
def to_valid_date(df):
    # 유효한 일자 형식
    valid_date_mask = (df['조사일자'].str.match(r'^\d{4}-\d{2}-\d{2}$') | 
                       df['조사일자'].str.match(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$') | 
                       df['조사일자'].str.match(r'^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$'))    
    # 유효한 일자만 선택
    cleaned_df = df.loc[valid_date_mask]
    cleaned_df.sort_values('조사일자', ascending=True, inplace=True)
    cleaned_df['조사일자'] = pd.to_datetime(cleaned_df['조사일자'], errors='coerce')
    return cleaned_df.dropna()

#### 데이터 통합

In [3]:
측정망_좌표 = pd.read_csv(f'{path}/해양환경측정망_조사정점.csv')
측정망_좌표.head()

,생태구역,정점명,개략위치 설명,위도,경도
0,서해중부 생태구,HC0139,인천항 중앙부,37.493889,126.617778
1,서해중부 생태구,HC0140,평택항 중앙부,36.959167,126.833333
2,서해중부 생태구,HC0424,대천항 중앙부,36.331667,126.508889
3,서남해역 생태구,HW0612,목포항 중앙부,34.785278,126.388611
4,서남해역 생태구,HW0613,여객선터미널 앞,34.778889,126.380556


In [4]:
col_names = [
    '년', '월', '조사일', '조사기간',
    '구분', '생태구', '해역', '정점명',
    '기상', '북위', '동경', '수심',
    '수질지수', '표층수온', '저층수온', '표층염분', '저층염분',
    '표층pH', '저층pH', '표층DO', '저층DO',
    '표층COD', '저층COD', '표층NH4-N', ' 저층NH4-N', '표층NO2-N',
    '저층NO2-N', '표층NO3-N', '저층NO3-N', '표층DIN','저층DIN',
    '표층T-N', '저층T-N', '표층DIP', '저층DIP', '표층T-P', '저층T-P',
    '표층SiO2-Si', '저층SiO2-Si', '표층SS', '저층SS', '표층chl', '저층Chl-a',
    '투명도', '기존연안명칭', 'St.'
]

해양환경측정망 = []
for file in [file for file in os.listdir(path) if file.endswith('.xlsx')]:
    for sheet_name in ('2월', '5월', '8월', '11월'):
        temp = pd.read_excel(f'{path}/{file}', sheet_name=sheet_name, header=2)
        temp.columns = col_names
        해양환경측정망.append(temp)
해양환경측정망 = pd.concat(해양환경측정망, axis=0)
해양환경측정망.drop(['St.', '북위', '동경'], axis=1, inplace=True)
해양환경측정망.head()

,년,월,조사일,조사기간,구분,생태구,해역,정점명,기상,수심,수질지수,표층수온,저층수온,표층염분,저층염분,표층pH,저층pH,표층DO,저층DO,표층COD,저층COD,표층NH4-N,저층NH4-N,표층NO2-N,저층NO2-N,표층NO3-N,저층NO3-N,표층DIN,저층DIN,표층T-N,저층T-N,표층DIP,저층DIP,표층T-P,저층T-P,표층SiO2-Si,저층SiO2-Si,표층SS,저층SS,표층chl,저층Chl-a,투명도,기존연안명칭
0,2011,2,16,14:36:00,하천영향 및 반폐쇄성해역(B),서해중부(C),한강하구,BC0101,맑음,11.0,III,1.21,1.19,29.15,29.17,8.013684,8.013684,12.370945,12.454083,4.065152,3.865292,182.100,175.000,10.472,11.914,180.950,204.582,373.522,391.496,876.54042,953.25062,6.169,10.943,100.246753,97.840831,160.132,184.324,79.6,89.2,4.524,3.6016,0.4,인천연안
1,2011,2,16,13:49:00,하천영향 및 반폐쇄성해역(B),서해중부(C),한강하구,BC0102,맑음,18.0,III,1.15,0.96,29.73,29.96,8.052871,8.082261,12.698314,12.442099,2.506244,2.626160,130.188,105.240,13.272,6.104,236.068,129.836,379.528,241.180,756.44170,569.01824,9.920,7.781,92.026519,78.493208,242.844,122.108,88.8,72.8,4.060,3.5960,0.4,인천연안
2,2011,2,16,16:07:00,하천영향 및 반폐쇄성해역(B),서해중부(C),한강하구,BC0103,맑음,10.0,III,1.13,1.01,29.89,30.01,8.043074,8.043074,12.051115,12.095622,2.466272,2.985908,163.172,155.274,11.550,6.790,218.862,147.392,393.584,309.456,767.49764,840.25144,14.849,16.058,74.884325,62.253234,228.172,144.200,40.0,46.4,3.596,10.7076,0.5,인천연안
3,2011,2,16,13:07:00,하천영향 및 반폐쇄성해역(B),서해중부(C),한강하구,BC0104,맑음,11.5,III,0.70,0.67,30.45,30.46,8.062667,8.082261,12.486000,12.439967,2.466272,2.586188,139.328,154.140,6.930,7.882,166.446,187.292,312.704,349.314,634.38676,591.48180,23.188,22.537,65.862117,61.351013,190.568,224.700,32.0,31.6,4.002,4.0020,0.5,인천연안
4,2011,2,16,11:43:00,하천영향 및 반폐쇄성해역(B),서해중부(C),한강하구,BC0105,맑음,19.5,III,0.74,0.60,30.36,30.48,7.974497,7.984294,12.219325,11.982240,1.506944,1.307084,174.342,146.566,7.182,7.868,168.476,189.896,350.000,344.330,634.23290,666.85122,8.587,12.710,64.959896,73.079883,184.100,223.832,14.0,41.2,2.204,4.4080,0.5,인천연안


In [5]:
해양환경측정망 = pd.merge(해양환경측정망, 측정망_좌표, how='left', on='정점명')
해양환경측정망.drop(columns='생태구역', inplace=True)
해양환경측정망.dropna(axis=0, inplace=True)

해양환경측정망['수질지수'] = 해양환경측정망['수질지수'].replace({'I':'1', 'II':'2', 'III':'3', 'IV':'4', 'V':'5'}).astype(int)

In [6]:
fix_input_errors(해양환경측정망, '표층pH', '8..21', 8.21)
fix_input_errors(해양환경측정망, '표층수온', '15..78', 15.78)
fix_input_errors(해양환경측정망, '저층염분', '34.2234.26', 34.22)
fix_input_errors(해양환경측정망, '저층pH', '8,15', 8.15)
fix_input_errors(해양환경측정망, '저층pH', '7,90', 7.90)

# 정수화
해양환경측정망['표층pH'] = 해양환경측정망['표층pH'].astype(int)
해양환경측정망['저층pH'] = 해양환경측정망['저층pH'].astype(int)

In [7]:
# 생태구 오입력 수정
해양환경측정망['생태구'] = list(map(lambda x: x.split('(')[0].rstrip(), 해양환경측정망['생태구']))

In [8]:
# 조사일자 추가
해양환경측정망 = date_ocean(해양환경측정망)

In [9]:
# 날짜 데이터 이상치 검증
invalid(해양환경측정망)

250      2011-02-23 1900-01-12 12:00:00
3770                  2013-11-15 13.:32
4320                  2014-02-05 13.:14
4172     2014-02-22 1900-01-11 02:24:00
4308                   2014-02-25 14.51
                      ...              
15627                  2020-11-25 11;05
16219                  2021-05-06 13;21
17003                  2021-08-04 12;32
17662    2022-02-12 1900-01-10 05:02:24
17888    2022-02-20 1900-01-08 09:07:12
Name: 조사일자, Length: 68, dtype: object

In [10]:
# 날짜 데이터 이상치 제거
해양환경측정망 = to_valid_date(해양환경측정망)

In [11]:
# 생태구 별 df 처리
대한해협 = 해양환경측정망.loc[해양환경측정망['생태구'] == '대한해협'].reset_index(drop=True)
서해중부 = 해양환경측정망.loc[해양환경측정망['생태구'] == '서해중부'].reset_index(drop=True)
동해 = 해양환경측정망.loc[해양환경측정망['생태구'] == '동해'].reset_index(drop=True)
서남해역 = 해양환경측정망.loc[해양환경측정망['생태구'] == '서남해역'].reset_index(drop=True)
제주 = 해양환경측정망.loc[해양환경측정망['생태구'] == '제주'].reset_index(drop=True)

In [12]:
# 수질지수 데이터 저장
ocean_rank(해양환경측정망).to_csv('./after_data/수질지수.csv', index=False, encoding='utf-8')

### 해역별 어업 데이터 (모델 학습 데이터)
#### 이업별_어선조업위치_데이터.csv
##### 컬럼명 설명
1. FS_NM / 어선명: OO호(어선명)
2. FS_NO / 어선번호: 0000000-0000000(어선번호)
3. FSH_PRMS_SCTN / 어업허가구분: 연안-1/근해-2/육상해수양식-3/종묘생산-4 (면허/허가/신고어업중 허가어업의 종류)
4. FSH_KIND / 어업종류: "연안자망/연안개량안강망/연안선망/연안통발/연안들망/연안조망/연안선인망/연안복합/근해채낚기/근해자망/근해안강망/근해봉수망/근해자리돔들망/근해장어통발/근해문어단지/근해통발/근해연승/근해형망/기선권현망/잠수기"
5. FS_LCT_LA / 어선위치위도: 어선위치위도 -도
6. FS_LCT_LO / 어선위치경도: 어선위치경도 -도
7. MRN_NO / 해구도번호: 해구도번호
8. FW_YR / 조업일자년: 조업일자-년
9. FW_MNTH / 조업일자월: 조업일자-월
#### 이업별_조업정보데이터.csv
##### 컬럼명 설명
1. FS_NM / 어선명: OO호(어선명)
2. FS_NO / 어선번호: 0000000-0000000(어선번호)
3. FSH_KIND / 어업종류: "연안자망/연안개량안강망/연안선망/연안통발/연안들망/연안조망/연안선인망/연안복합/근해채낚기/근해자망/근해안강망/근해봉수망/근해자리돔들망/근해장어통발/근해문어단지/근해통발/근해연승/근해형망/기선권현망/잠수기"
4. BUOY_KND_NM / 어구종류이름: 현재조업하는 어구 종류 이름/어구도감참조
5. SOF_NM / 조업대상어종이름: 조업대상 어종이름
6. FSH_COLCT_CNT / 어구양망횟수: 어구 양망 횟수
7. FSHNG_INTRVL / 어구사용간격: 어구도감의 어종별 어구별 조업시간
8. FW_YR / 조업일자년: 조업일자-년
9. FW_MNTH / 조업일자월: 조업일자-월
10. FW_DAY / 조업일자일: 조업일자-일
11. FW_HMS / 조업일자시간: 조업일자-시간
12. MRN_NO / 해구도번호: 해구도번호
13. OBSV_HMS / 관측시간: 관측 시간(YYYY-MM-DD HH:MM:SS)
14. CRRNT_SPD / 유속: 유속 값(cm/s)
15. CRRNT_DRCN / 유향: 유향 값(deg)

#### 데이터 통합

In [13]:
어선조업 = pd.read_csv(f'{path}/어업별+어선조업+위치.csv').dropna(axis=0)
조업정보 = pd.read_csv(f'{path}/어업별+조업+정보.csv')

1. FS_NM / 어선명: OO호(어선명)
2. FS_NO / 어선번호: 0000000-0000000(어선번호)
3. FSH_KIND / 어업종류: "연안자망/연안개량안강망/연안선망/연안통발/연안들망/연안조망/연안선인망/연안복합/근해채낚기/근해자망/근해안강망/근해봉수망/근해자리돔들망/근해장어통발/근해문어단지/근해통발/근해연승/근해형망/기선권현망/잠수기"
4. BUOY_KND_NM / 어구종류이름: 현재조업하는 어구 종류 이름/어구도감참조
5. SOF_NM / 조업대상어종이름: 조업대상 어종이름
6. FSH_COLCT_CNT / 어구양망횟수: 어구 양망 횟수
7. FSHNG_INTRVL / 어구사용간격: 어구도감의 어종별 어구별 조업시간
8. FW_YR / 조업일자년: 조업일자-년
9. FW_MNTH / 조업일자월: 조업일자-월
10. FW_DAY / 조업일자일: 조업일자-일
11. FW_HMS / 조업일자시간: 조업일자-시간
12. MRN_NO / 해구도번호: 해구도번호
13. OBSV_HMS / 관측시간: 관측 시간(YYYY-MM-DD HH:MM:SS)
14. CRRNT_SPD / 유속: 유속 값(cm/s)
15. CRRNT_DRCN / 유향: 유향 값(deg)

In [14]:
# 컬럼명 한글화 및 데이터 타입 변경
어선조업.columns = ['어선명', '어선번호', '어업허가구분', '어업종류', '조업_위도', '조업_경도', '해구도번호', '년', '월']
조업정보.columns = ['어선명', '어선번호', '어업종류', '어구종류', '어종', '어구양망횟수', '어구사용간격', '년', '월', '일', '시간', '해구도번호', '관측시간', '유속', '유향']
어선조업['해구도번호'] = 어선조업['해구도번호'].astype(int)
조업정보['해구도번호'] = 조업정보['해구도번호'].astype(int)
어선조업['년'] = 어선조업['년'].astype(int).astype(str)
어선조업['월'] = 어선조업['월'].astype(int).astype(str)
조업정보['년'] = 조업정보['년'].astype(int).astype(str)
조업정보['월'] = 조업정보['월'].astype(int).astype(str)

In [15]:
# 위도 경도 
display(어선조업.groupby('해구도번호')['조업_위도'].agg(['min', 'max']).reset_index())
display(어선조업.groupby('해구도번호')['조업_경도'].agg(['min', 'max']).reset_index())

,해구도번호,min,max
0,97,34.325590,34.730786
1,104,34.060249,34.513894
2,193,33.279145,35.291859
3,202,33.260774,35.279544
4,203,33.228615,35.378465
5,204,34.961359,35.051128
6,208,34.259563,34.259563
7,209,34.257988,34.257988
8,210,34.262466,34.262466
9,211,33.228360,35.378465


,해구도번호,min,max
0,97,127.508003,1277.513900
1,104,127.443642,127.930382
2,193,121.988838,126.916042
3,202,121.738206,126.914247
4,203,121.691523,126.916042
5,204,126.226407,126.358926
6,208,124.257640,124.257640
7,209,124.758710,124.758710
8,210,125.263030,125.263030
9,211,121.749193,126.916736


In [16]:
# 입력오류 처리
어선조업['조업_위도'].replace({0.743123 : 33.743123}, inplace=True)
어선조업['조업_경도'].replace({16.627274 : 126.627274}, inplace=True)
어선조업['조업_경도'].replace({1277.513900 : 127.513900}, inplace=True)

In [17]:
# 해구도별 평균 좌표 DataFrame 생성
해구도_좌표 = 어선조업.groupby(by='해구도번호', as_index=False).agg(
    조업_위도=('조업_위도', 'mean'),
    조업_경도=('조업_경도', 'mean')
)

# 어선명별 평균 좌표 DataFrame 생성
어선명_좌표 = 어선조업.groupby(by='어선명', as_index=False).agg(
    조업_위도=('조업_위도', 'mean'),
    조업_경도=('조업_경도', 'mean')
)

In [18]:
# 조업정보 데이터에 좌표 컬럼 추가
조업정보_좌표추가1 = pd.merge(조업정보, 어선명_좌표, on='어선명', how='left')
조업정보_좌표추가2 = pd.merge(조업정보_좌표추가1[조업정보_좌표추가1['조업_위도'].isna()].drop(columns=['조업_위도', '조업_경도']), 해구도_좌표, on='해구도번호', how='left')
조업정보_re = pd.concat([조업정보_좌표추가1.dropna(subset=['조업_위도', '조업_경도']), 조업정보_좌표추가2], axis=0, join='outer')
조업정보.shape, 조업정보_re.shape

((57189, 15), (57189, 17))

In [19]:
# 어종 입력 오류 처리
조업정보_re['어종'].replace({'걀치':'갈치', '고동':'고둥', '정어':'정어리', '부새':'부세'}, inplace=True)
조업정보_re.drop('어구종류', axis=1, inplace=True)

In [20]:
# 일자 컬럼 생성 및 불필요 컬럼 삭제
조업정보_re['일자'] = 조업정보_re['년'] + '-' + 조업정보_re['월'].str.zfill(2) + '-' + 조업정보_re['일'].str.zfill(2) + ' ' + 조업정보_re['시간'].str.zfill(5)
조업정보_re['일자'] = pd.to_datetime(조업정보_re['일자'], errors='coerce')

In [21]:
조업일_어종_조업정보 = pd.DataFrame()

for fish in 조업정보_re['어종'].unique():
    df = 조업정보_re[조업정보_re['어종'] == fish]
    df['일자'] = df['일자'].dt.date
    # 조업일자, 조업대상어종별 어획량 및 위도/경도 범위 계산
    df_info = df.groupby(['일자', '어종'], as_index=True)[['어구양망횟수', '조업_위도', '조업_경도']].agg(
        개체수=('어구양망횟수', 'count'),
        조업_위도=('조업_위도', 'mean'),
        조업_경도=('조업_경도', 'mean'),
    ).reset_index()
    # 결과를 하나의 DataFrame에 누적
    조업일_어종_조업정보 = pd.concat([조업일_어종_조업정보, df_info], ignore_index=True)
조업일_어종_조업정보.shape

(7673, 5)

In [22]:
# 조업정보 및 개체수 데이터 병합
left = 조업정보_re.copy()
right = 조업일_어종_조업정보.copy().drop(columns=['조업_위도', '조업_경도'])
left['temp'] = left['일자'].dt.date
right['temp'] = right['일자']
right.drop(columns='일자', inplace=True)
df = pd.merge(left, right, 'left', ['어종','temp'])

# 결측치 및 미사용 컬럼 삭제
df = df.drop(columns=['temp', '년', '월', '일', '시간', '어선명', '어선번호', '어구양망횟수', '어구사용간격', '관측시간', '유속', '유향']).dropna(axis=0).reset_index(drop=True)

# 개체수 결측치 제거 및 형변환
df = df[df['개체수']!=0].reset_index(drop=True)
df['개체수'] = df['개체수'].astype(int)

In [23]:
# 데이터 병합
left = df.copy()
left['temp'] = left['일자'].dt.date

# 대한해협 어업 데이터 저장
대한해협['temp'] = 대한해협['조사일자'].dt.date
pd.merge(left, 대한해협, 'left', 'temp').drop(columns=['temp', '조사기간', '해역']).dropna(axis=0).to_csv('./model_data/대한해협_어업.csv', index=False)

# 동해 어업 데이터 저장
동해['temp'] = 동해['조사일자'].dt.date
pd.merge(left, 동해, 'left', 'temp').drop(columns=['temp', '조사기간', '해역']).dropna(axis=0).to_csv('./model_data/동해_어업.csv', index=False)

# 서남해역 어업 데이터 저장
서남해역['temp'] = 서남해역['조사일자'].dt.date
pd.merge(left, 서남해역, 'left', 'temp').drop(columns=['temp', '조사기간', '해역']).dropna(axis=0).to_csv('./model_data/서남해역_어업.csv', index=False)

# 서해중부 어업 데이터 저장
서해중부['temp'] = 서해중부['조사일자'].dt.date
pd.merge(left, 서해중부, 'left', 'temp').drop(columns=['temp', '조사기간', '해역']).dropna(axis=0).to_csv('./model_data/서해중부_어업.csv', index=False)

# 제주 어업 데이터 저장
제주['temp'] = 제주['조사일자'].dt.date
pd.merge(left, 제주, 'left', 'temp').drop(columns=['temp', '조사기간', '해역']).dropna(axis=0).to_csv('./model_data/제주_어업.csv', index=False)

### 기상청 표층수온/표층염분 예측치 데이터 전처리

#### 데이터 통합

In [31]:
import netCDF4 as nc
nc.Dataset('./old_data/AR6_SSP585_ENSMN_korea_SALNT_gridraw_monthly_2021_2040.nc')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.5
    dimensions(sizes): time(240), lat(170), lon(140)
    variables(dimensions): float64 SALNT(time, lat, lon), int64 time(time), float64 lat(lat), float64 lon(lon)
    groups: 

In [36]:
xr.open_dataset('./old_data/AR6_SSP585_ENSMN_korea_SALNT_gridraw_monthly_2021_2040.nc')

OSError: [Errno 22] Invalid argument: 'c:\\Users\\User\\Desktop\\에이블스쿨\\빅프\\DX_빅프로젝트\\25조\\주 산출물\\최종\\DX 25조 개발코드\\AIVLE\\old_data\\AR6_SSP585_ENSMN_korea_SALNT_gridraw_monthly_2021_2040.nc'

In [26]:
salnt = []
sst = []
for file in [file for file in os.listdir(path) if file.endswith('nc')]:
    if 'SALNT' in file:
        salnt.append(xr.open_dataset(f'{path}/{file}').to_dataframe().reset_index())
    else:
        sst.append(xr.open_dataset(f'{path}/{file}').to_dataframe().reset_index())

salnt = pd.concat(salnt, axis=0)
sst = pd.concat(sst, axis=0)

data = pd.merge(sst, salnt, on=['time', 'lat', 'lon'])
data.rename({'time':'기간', 'lat':'위도', 'lon':'경도', 'SALNT':'표층염분', 'SST':'표층수온'}, axis=1, inplace=True)
data.head()

AR6_SSP585_ENSMN_korea_SALNT_gridraw_monthly_2021_2040.nc


OSError: [Errno 22] Invalid argument: 'c:\\Users\\User\\Desktop\\에이블스쿨\\빅프\\DX_빅프로젝트\\25조\\주 산출물\\최종\\DX 25조 개발코드\\AIVLE\\old_data\\AR6_SSP585_ENSMN_korea_SALNT_gridraw_monthly_2021_2040.nc'

#### 데이터 전처리(이상치 제거)

- 표층염도 이상치 제거
- 세계 바다 염분 농도의 평균은 34.72 psu
- 태평양 바다 염분 농도의 평균은 34.62 psu
- 현재 동해 평균 염분 농도: 34.5 psu
- 현재 남해 평균 염분 농도: 34 psu
- 현재 서해 평균 염분 농도: 33 psu
- 세계 바다 염분 농도의 평균과 현재 한국 근해의 염분 농도를 참조하여
- 기상청 예측 모델의 미래 염분 예측치가 30 미만인 것은 이상치라고 판단하여 배제함

In [ ]:
df = data.loc[data['표층염분']>=30, :].reset_index(drop=True)
df.head()

In [ ]:
# 데이터 크기를 줄이기 위해 5년치 데이터만 추출
df['기간'] = pd.to_datetime(df['기간'], format='%Y-%m-%d')
df = df[df['기간'].between('2025-01-31', '2030-12-31', inclusive='both')].reset_index(drop=True)
df.head()

In [ ]:
df.to_csv('./after_data/표층수온_표층염도_예측치.csv', encoding='utf-8', index=False)